In [ ]:
!pip install scikit-learn
import pandas as pd

In [ ]:
df = pd.read_csv('sepsis_df.csv')

In [ ]:
df.columns

In [ ]:
# Rename columns that start with 'latest_intubation' to start with 'time_zero'
df.rename(
    columns={col: col.replace('latest_intubation', 'time_4h', 1) 
             for col in df.columns if col.startswith('latest_intubation')},
    inplace=True
)

In [ ]:
# Evaluate mechanical power at 0 and 72 hours
mp_0 = df['mechanical_power_0']
mp_72 = df['mechanical_power_72hr']

print("Mechanical Power at 0 hours:")
print(mp_0.describe())

print("\nMechanical Power at 72 hours:")
print(mp_72.describe())

In [ ]:
# Define outliers as values outside 1.5*IQR from Q1 and Q3 for both timepoints

def count_outliers(series):
    q1 = series.quantile(0.25)
    q3 =  series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return ((series < lower) | (series > upper)).sum()

outliers_0 = count_outliers(mp_0)
outliers_72 = count_outliers(mp_72)

print(f"Number of outliers at 0 hours: {outliers_0}")
print(f"Number of outliers at 72 hours: {outliers_72}")

In [ ]:
# Remove entire rows where mechanical_power_0 or mechanical_power_72hr are outliers

q1_0 = mp_0.quantile(0.25)
q3_0 = mp_0.quantile(0.75)
iqr_0 = q3_0 - q1_0
lower_0 = q1_0 - 1.5 * iqr_0
upper_0 = q3_0 + 1.5 * iqr_0

q1_72 = mp_72.quantile(0.25)
q3_72 = mp_72.quantile(0.75)
iqr_72 = q3_72 - q1_72
lower_72 = q1_72 - 1.5 * iqr_72
upper_72 = q3_72 + 1.5 * iqr_72

# Identify outlier rows for either timepoint
outlier_rows = (
    (df['mechanical_power_0'] < lower_0) | (df['mechanical_power_0'] > upper_0) |
    (df['mechanical_power_72hr'] < lower_72) | (df['mechanical_power_72hr'] > upper_72)
)

# Remove the entire rows where these issues arise
df = df[~outlier_rows]

In [ ]:
df = pd.get_dummies(df, columns=['gender'], prefix='gender')

In [ ]:
from sklearn.model_selection import train_test_split

# Use 'tracheostomy_flag' as the outcome variable
X = df[['cumulative_vent_days', 
'age', 'gender_F', 'gender_M',
       'mechanical_power_0', 'mechanical_power_72hr']]
y = df['tracheostomy_flag']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
validation = pd.read_csv('kmimic_final.csv')
print(len(validation))
# Select and copy relevant columns to avoid SettingWithCopyWarning
val = validation[['cumulative_vent_days', 'age', 'gender_F', 'gender_M', 'mech_power_4h', 'mech_power_72h', 'trach_flag']].copy()

# Create new columns based on existing ones
val['mechanical_power_0'] = val['mech_power_4h']
val['mechanical_power_72hr'] = val['mech_power_72h']
val['tracheostomy_flag'] = val['trach_flag']
# x_y
val_X = val[['cumulative_vent_days', 'age', 'gender_F', 'gender_M', 'mechanical_power_0', 'mechanical_power_72hr']]
val_y = val['tracheostomy_flag']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score, accuracy_score, recall_score,
    classification_report, confusion_matrix
)

# Initialize Random Forest with best parameters
best_rf = RandomForestClassifier(
    criterion='entropy',
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=50,
    random_state=42
)

# Train the model
best_rf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = best_rf.predict(X_test)

# Evaluate
print("Random Forest - Test set accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest - Test set precision:", precision_score(y_test, y_pred_rf))
print("Random Forest - Test set recall:", recall_score(y_test, y_pred_rf))
print("\nRandom Forest - Classification Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest - Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import precision_score, roc_auc_score, roc_curve, accuracy_score
import numpy as np
import matplotlib.pyplot as plt  # <-- Fix: import plt

# Define parameter grid for randomized search
param_dist = {
    'learning_rate': [0.05, 0.1],
    'max_iter': [100],
    'max_depth': [3, None],
    'min_samples_leaf': [10],
    'l2_regularization': [0.01, 0.1]
}

# Set up cross-validation: 5 folds, 10 repeats
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

# Set up the randomized search, optimizing for precision
search = RandomizedSearchCV(
    HistGradientBoostingClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=30,
    scoring='precision',
    n_jobs=-1,
    cv=cv,
    verbose=1,
    random_state=42
)

# Fit randomized search
search.fit(X_train, y_train)

print("Best parameters found:", search.best_params_)
print(f"Best cross-validated precision: {search.best_score_:.3f}")

# Use the best estimator to predict
best_skxgb = search.best_estimator_
y_pred_skxgb = best_skxgb.predict(X_test)
y_pred_proba_skxgb = best_skxgb.predict_proba(X_test)[:, 1]

# Evaluate performance
auc_skxgb = roc_auc_score(y_test, y_pred_proba_skxgb)
acc_skxgb = accuracy_score(y_test, y_pred_skxgb)
prec_skxgb = precision_score(y_test, y_pred_skxgb)
print(f"sklearn HistGradientBoosting ROC AUC: {auc_skxgb:.3f}")
print(f"sklearn HistGradientBoosting Accuracy: {acc_skxgb:.3f}")
print(f"sklearn HistGradientBoosting Precision: {prec_skxgb:.3f}")

# Plot ROC curve
fpr_skxgb, tpr_skxgb, _ = roc_curve(y_test, y_pred_proba_skxgb)
plt.figure(figsize=(8, 6))
plt.plot(fpr_skxgb, tpr_skxgb, label=f'sklearn HistGradientBoosting (AUC = {auc_skxgb:.2f})', color='navy')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - sklearn HistGradientBoosting')
plt.legend(loc="lower right")
plt.show()


In [ ]:
y_val_predict = best_rf.predict(val_X)
y_val_pred_proba = best_rf.predict_proba(val_X)[:, 1]

# Evaluate performance
auc_rf = roc_auc_score(val_y, y_val_pred_proba)
acc_rf = accuracy_score(val_y, y_val_predict)
prec_rf = precision_score(val_y, y_val_predict)
# print(f"sklearn RandomForest ROC AUC: {auc_rf:.3f}")
# print(f"sklearn RandomForest Accuracy: {acc_rf:.3f}")
# print(f"sklearn RandomForest Precision: {prec_rf:.3f}")

from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Compute ROC curve for Random Forest
fpr_rf, tpr_rf, _ = roc_curve(val_y, y_val_pred_proba)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest ROC (AUC = {auc_rf:.2f})', color='forestgreen')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Validation ROC Curve - Random Forest')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import precision_score, roc_auc_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt

# Initialize model with best parameters
best_skxgb = HistGradientBoostingClassifier(
    min_samples_leaf=10,
    max_iter=100,
    max_depth=None,
    learning_rate=0.1,
    l2_regularization=0.01,
    random_state=42
)

# Fit the model
best_skxgb.fit(X_train, y_train)

# Predict
y_pred_skxgb = best_skxgb.predict(X_test)
y_pred_proba_skxgb = best_skxgb.predict_proba(X_test)[:, 1]

# Evaluate performance
auc_skxgb = roc_auc_score(y_test, y_pred_proba_skxgb)
acc_skxgb = accuracy_score(y_test, y_pred_skxgb)
prec_skxgb = precision_score(y_test, y_pred_skxgb)
print(f"sklearn HistGradientBoosting ROC AUC: {auc_skxgb:.3f}")
print(f"sklearn HistGradientBoosting Accuracy: {acc_skxgb:.3f}")
print(f"sklearn HistGradientBoosting Precision: {prec_skxgb:.3f}")

# Plot ROC curve
fpr_skxgb, tpr_skxgb, _ = roc_curve(y_test, y_pred_proba_skxgb)
plt.figure(figsize=(8, 6))
plt.plot(fpr_skxgb, tpr_skxgb, label=f'sklearn HistGradientBoosting (AUC = {auc_skxgb:.2f})', color='navy')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - sklearn HistGradientBoosting')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_val_predict = best_skxgb.predict(val_X)
y_val_pred_proba = best_skxgb.predict_proba(val_X)[:, 1]

# Evaluate performance
auc_skxgb = roc_auc_score(val_y, y_val_pred_proba)
acc_skxgb = accuracy_score(val_y, y_val_predict)
prec_skxgb = precision_score(val_y, y_val_predict)
print(f"sklearn HistGradientBoosting ROC AUC: {auc_skxgb:.3f}")
print(f"sklearn HistGradientBoosting Accuracy: {acc_skxgb:.3f}")
print(f"sklearn HistGradientBoosting Precision: {prec_skxgb:.3f}")

from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Compute ROC curve
fpr, tpr, _ = roc_curve(val_y, y_val_pred_proba)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Validation ROC (AUC = {auc_skxgb:.2f})', color='darkorange')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Validation ROC Curve - HistGradientBoosting')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('a.png')

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, accuracy_score
import matplotlib.pyplot as plt

# HistGradientBoosting predictions on validation set
y_val_pred_skxgb = best_skxgb.predict(val_X)
y_val_proba_skxgb = best_skxgb.predict_proba(val_X)[:, 1]

# RandomForest predictions on validation set
y_val_pred_rf = best_rf.predict(val_X)
y_val_proba_rf = best_rf.predict_proba(val_X)[:, 1]

# ROC metrics
fpr_skxgb, tpr_skxgb, _ = roc_curve(val_y, y_val_proba_skxgb)
auc_skxgb = roc_auc_score(val_y, y_val_proba_skxgb)

fpr_rf, tpr_rf, _ = roc_curve(val_y, y_val_proba_rf)
auc_rf = roc_auc_score(val_y, y_val_proba_rf)

# Plot both ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_skxgb, tpr_skxgb, label=f'HistGradientBoosting (AUC = {auc_skxgb:.2f})', color='navy')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.2f})', color='forestgreen')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Validation Set')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()